#### Imports

In [1]:
# Nuestro path de inicio siempre debe ser el home dir, e
import os
os.chdir(os.path.abspath(".."))

# Imports
import json
import pandas as pd
import stable_whisper
import warnings
from src.dataloaders import cargar_audio_data

# ignoramos warnings
warnings.filterwarnings('ignore')

#### Files

In [2]:
# Archivos importantes y paths
df_annotations = pd.read_excel('data/annotations.xlsx')

# Omitimos todos los audios en development
df_annotations = df_annotations[df_annotations['Type'] != 'Development'].reset_index(drop = True)

In [3]:
# Obtenemos el json con los intervalos
with open('data/objetivos_por_voto_promedio_whisper.json', 'r') as f:
    targets_mean_vote = json.load(f)
    
# Comprabamos seguimos teniendo los 244 audios
len(targets_mean_vote.keys())

205

#### Loop para extracción de data cruda de los audios

In [4]:
# Audios que no estan en development
lista_audios = df_annotations['Audio_Name'].unique()
len(lista_audios)

205

In [5]:
for audio_name in lista_audios:
    
    print(f'Procesando {audio_name}')
    data, time, sr = cargar_audio_data(df_annotations = df_annotations, audio_name = audio_name)
    
    assert len(data) == len(time)
    
    df_audio = pd.DataFrame(data = data, columns = ['Data'])
    df_audio['Time'] = time
    
    x, y, z = [],[],[]
    for inter, ind in zip(targets_mean_vote[audio_name]['rangos'], targets_mean_vote[audio_name]['indice']):
        inicio = inter[0]
        fin = inter[1]
        x.append(tuple(df_audio[(df_audio['Time'] >= inicio) & (df_audio['Time'] <= fin)]['Data'].values.tolist()))
        y.append((inicio,fin))
        z.append(ind)
        
    df_audio = pd.DataFrame()
    df_audio['Data'] = x
    df_audio['Time'] = y
    df_audio['Indice'] = z
    
    df_audio.to_csv(f'data/FEATURES/RAW/{audio_name}.csv', index = False)
    print(f'{audio_name} procesado con éxito')
    print('')
    print('---------------------------------------------------------') 
    break

Procesando MSP-Conversation_0021.wav
MSP-Conversation_0021.wav procesado con éxito

---------------------------------------------------------


In [19]:
asd = pd.DataFrame()

In [20]:
asd['a'] = x

In [21]:
asd

,a
0,"[1.4335335, 0.07602339, 0.7426901, 0.35589093,..."
1,"[1.4006708, 0.15309446, 0.81433225, 0.3753919,..."
